In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Index

1 - API data\
2 - Data preprocessing\
3 - 

In [ ]:
## APi Key to arg parse

In [ ]:
summoner = 'Saikki kusuo'
key = None

Summoner v4, /lol/summoner/v4/summoners/by-name/{summonerName}\
Get summoner info by name using API key and summoner name

In [ ]:
# Summoner v4, /lol/summoner/v4/summoners/by-name/{summonerName}
# Get summoner info by name using API key and summoner name

def get_accountid(summoner_name, apikey):
    # This function takes the summoner name and the API Key
    # and retrieves its account id
    url = f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    accid = json['accountId']
    
    return accid

In [ ]:
accountId = get_accountid(summoner, key)
accountId

In [ ]:
def get_matchlist(accountid, api):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    
    return json

In [ ]:
mygames = get_matchlist(accountId, key)

In [ ]:
mygames.keys()

In [ ]:
mygames = pd.DataFrame(mygames['matches'])
mygames.head()

In [ ]:
# Match v4, /lol/match/v4/matches/{matchId}
# Get match info using gameid
def get_match_info(matchid):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matches/{matchid}'
    html = requests.get(url,
                    params={'api_key': key})
    json = html.json()
    
    return json

In [ ]:
match = get_match_info(4834424471)

In [ ]:
match.keys()

In [ ]:
match['participants']

In [ ]:
match['participantIdentities'][0]

In [ ]:
players = pd.DataFrame()

for participant in range(10):
    df = pd.DataFrame.from_dict(match['participants'][participant]['stats'], orient='index').T
    df.insert(loc=0, column='championId', value= match['participants'][participant]['championId'])
    df.insert(loc=1, column='teamId', value=match['participants'][participant]['teamId'])
    players = players.append(df)
    
players.insert(loc=0, column='gameid', value=[match['gameId'] for value in range(len(players))])

In [ ]:
players

In [ ]:
players.columns.to_list()

In [ ]:
player_info = pd.DataFrame()

for elem in range(len(match['participantIdentities'])):
    df = pd.DataFrame.from_dict(match['participantIdentities'][elem]['player'], orient='index').T
    df.insert(loc=0, column='participantId', value= match['participantIdentities'][elem]['participantId'])
    player_info = player_info.append(df)

In [ ]:
player_info

In [ ]:
merge = pd.merge(players, player_info, on='participantId')

In [ ]:
merge

In [ ]:
merge.columns.to_list()

## Game timeline

In [ ]:
def get_match_timeline(matchid):

    url= f'https://euw1.api.riotgames.com/lol/match/v4/timelines/by-match/{matchid}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    
    return json

In [ ]:
tl = get_match_timeline(4834424471)

tl.keys()

In [ ]:
tl['frames'][0].keys()

In [ ]:
len(tl)

In [ ]:
tl['frames'][35]['timestamp']

In [ ]:
it = tl['frames'][16]['participantFrames']

a = pd.DataFrame(columns=['participantId', 'x', 'y' 'currentGold', 'totalGold', 'level', 'xp',
                          'minionsKilled', 'jungleMinionsKilled', 'dominionScore', 'teamScore'])
it

In [ ]:

for elem in it:
    dic = {}
    for sub_elem in it[elem]:         
        if type(it[elem][sub_elem]) == dict:
            dic.update(it[elem][sub_elem])
        else:
            dic[sub_elem] = it[elem][sub_elem]
    a = a.append(pd.DataFrame(dic, index=[0]))


In [ ]:
a

In [ ]:
def participant_frames(json):
    participant_frames = pd.DataFrame(columns=['participantId', 'x', 'y' 'currentGold', 'totalGold', 'level', 'xp',
                          'minionsKilled', 'jungleMinionsKilled', 'dominionScore', 'teamScore', 'timestamp'])
    
    lenght = list(range(len(json['frames'])))
    for frame in lenght:
        iterable = json['frames'][frame]['participantFrames']
        for elem in iterable:
            dic = {}
            for sub_elem in iterable[elem]:         
                if type(iterable[elem][sub_elem]) == dict:
                    dic.update(iterable[elem][sub_elem])
                else:
                    dic[sub_elem] = iterable[elem][sub_elem]
            df = pd.DataFrame(dic, index=[0])
            df.insert(loc=0, column='timestamp', value=json['frames'][frame]['timestamp'])
            participant_frames = participant_frames.append(df, ignore_index=True)
    return participant_frames

In [ ]:
a = participant_frames(tl)

a['dominionScore'].unique()

In [ ]:
tl['frames'][35]

In [ ]:
tl['frames'][1]['timestamp']

In [ ]:
tl['frames'][1]['events']

In [ ]:
tl

In [ ]:
events = pd.DataFrame(tl['frames'][4]['events'])

In [ ]:
events = pd.DataFrame()
frames = tl['frames']
for elem in range(len(frames)):
    events = events.append(pd.DataFrame(frames[elem]['events']))
                           
events 

In [ ]:
events['type'].unique()

## OOP version

In [ ]:
class User(object):  
        
    
    def __init__(self, # self makes reference to the instance
                 name, 
                 api_key, 
                 summoner_name,    
                 ):
        
        self.name = name
        self.api_key = api_key
        self.summoner_name = summoner_name
        self.accountid = get_accountid(summoner, key)
     
    
    def new_apikey(self, key):
        self.api_key = key
        

In [ ]:
events['position_x'] = [elem.get('x') if type(elem) == dict else 'none' for elem in events['position']]
events['position_y'] = [elem.get('y') if type(elem) == dict else 'none' for elem in events['position']]
events.drop(columns='position', inplace=True)

In [ ]:
events.head()

In [ ]:
events['assistingParticipantIds'][2]

In [ ]:
events['assistingParticipantIds'] = [elem[0] if type(elem) == list else 'none' for elem in events['assistingParticipantIds']]

In [ ]:
events

In [ ]:
events.dtypes

In [ ]:
num_feat = events.select_dtypes(include=np.number).columns.to_list()
cat_feat = events.select_dtypes(include='object').columns.to_list()
num_df = events[num_feat]
cat_df = events[cat_feat].astype('category')

In [ ]:
events = pd.concat([cat_df, num_df], axis=1)
events.dtypes

In [ ]:
event = events[['type', 'wardType','assistingParticipantIds']]
event

In [ ]:
event.to_sql('events', con=engine, if_exists='append', index_label='id')

In [ ]:
from DB import engine